# **Contextual RAG**

文脈検索拡張生成（RAG）は、検索プロセス中に文脈圧縮を取り入れることで応答の関連性と効率性を向上させる高度なRAG技術です。従来のRAGでは、生成モデルに全文書が検索され、送信されますが、その中には関連性の低い情報も含まれている可能性があり、コストが高くなり、応答の精度が低下する可能性があります。

文脈型RAG(Contextual RAG)では、検索されたドキュメントは言語モデルに渡される前にドキュメント圧縮機能によって処理されます。この圧縮機能は、クエリに対して最も関連性の高い情報のみを抽出して保持し、関連性のないドキュメントはすべて破棄します。このアプローチにより、検索された文脈のノイズが低減され、結果として生成モデルからより正確で簡潔な、費用対効果の高い応答が返されることになります。

Reference: [Contextual RAG](https://python.langchain.com/docs/how_to/contextual_compression/)

In [2]:
# set apikey to environment
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

In [1]:
# load japanese data from PDF
from langchain_community.document_loaders import PyPDFLoader
file_path = "../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load_and_split()
documents[0]

Document(metadata={'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf', 'page': 0, 'page_label': '1'}, page_content='スタートアップの力で\n社会課題解決 と経済成長 を加速する\nスタートアップ育成に向けた政府の取組\n2024年9月')

In [3]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# split pages content
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

# load vectorstore
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)

# create retriever
retriever = vectorstore.as_retriever() # あとで baseのretrieverとして使う

# Contextual Retriever

In [4]:
# create llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

# create compression retriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [5]:
# checkimg compressed doc
compressed_docs = compression_retriever.invoke("スタートアップによる経済的効果は?")
compressed_docs

[Document(metadata={'page': 3, 'page_label': '4', 'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf'}, page_content='スタートアップによるGDP創出額は、直接効果で10.47兆円、間接波及効果を含めると19.39兆円と試算。\n直接効果は通信・放送業の名目GDPに、間接波及効果を含めた値は北海道内の名目GDPに相当しており、\n一定の経済的インパクトを発揮していると言える。'),
 Document(metadata={'page': 1, 'page_label': '2', 'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf'}, page_content='スタートアップは、経済成長のドライバー。将来の所得や財政を支える新たな担い手。\nスタートアップは、雇用創出にも大きな役割。\nスタートアップは、新たな社会課題を解決する主体としても重要。')]

# RAG Chain

In [6]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the following context.
If you don't find the answer in the context, just say that you don't know.
Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": compression_retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
# checking response
response = rag_chain.invoke("スタートアップによる経済的効果は?")
response

'スタートアップによる経済的効果は、直接効果で10.47兆円、間接波及効果を含めると19.39兆円と試算されています。'

# Evaluation

In [11]:
# prepare data for evaluation
def evaluate_rag_chain(rag_chain, retriever, file_path):
    """
    RAGチェーンの評価を行い、評価結果のDataFrameを返します。
    
    Args:
        rag_chain: 評価対象のRAGチェーン
        retriever: 使用するretriever
        file_path: 評価対象のファイルパス
    
    Returns:
        pd.DataFrame: 評価結果を含むDataFrame
    """
    # データセットの読み込みとフィルタリング
    from datasets import load_dataset
    import pandas as pd
    
    test_data = load_dataset("allganize/RAG-Evaluation-Dataset-JA")["test"]
    
    # 評価対象ファイル名の抽出
    tgt_name = file_path.split("file_name=")[-1]
    
    # 評価対象データのフィルタリング
    filtered_data = test_data.filter(lambda x: x["target_file_name"] == tgt_name)
    eval_df = filtered_data.to_pandas()[["question", "target_answer"]]
    
    # RAGチェーンを使用して回答と文脈を取得
    p_data = []
    for _, row in eval_df.iterrows():
        q = row["question"]
        t = row["target_answer"]
        
        # RAGチェーンを実行
        response_text = rag_chain.invoke(q)
        context_docs = retriever.invoke(q)
        
        p_data.append({
            "question": q,
            "target_answer": t,
            "response": response_text,
            "context": context_docs
        })
    
    # 結果をDataFrameに変換
    rag_eval_df = pd.DataFrame(p_data)
    
    return rag_eval_df

In [12]:
# 評価用データ作成実行
rag_eval_df = evaluate_rag_chain(
    rag_chain=rag_chain,
    retriever=compression_retriever,
    file_path=file_path
)

# 評価結果の確認
print(f"評価対象サンプル数: {len(rag_eval_df)}")
rag_eval_df.head()

Filter: 100%|██████████| 300/300 [00:00<00:00, 6260.44 examples/s]


評価対象サンプル数: 7


,question,target_answer,response,context
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,I don't know.,[page_content='オープンイノベーション促進税制において、スタートアップ企業の株...
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,具体的な条件として、企業が主に「国内で」、「自ら」開発した知的財産権（特許権等、AI関連のプ...,[page_content='イノベーション拠点税制における所得控除について、企業が主に「国...
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,「カーブアウト加速等支援事業」の主な目的は、事業会社に蓄積されている技術を活用し、新たな会社...,[page_content='「カーブアウト加速等支援事業」' metadata={'pag...
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,"I'm sorry, I don't know.",[]
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,"I'm sorry, but the specific information about ...",[page_content='- 2023年１月、産業革新投資機構（JIC）の子会社であるベ...


In [14]:
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

relevance_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="relevance",
    model="gpt-4o-mini",
    model_kwargs={"temperature": 0}
)

accuracy_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="accuracy",
    model="gpt-4o-mini",
    model_kwargs={"temperature": 0}
)

def evaluate_response(df):
    """
    与えられたデータリストに対する Relevance と Accuracy の評価を行い、結果を DataFrame で返します。
    """
    def _flatten_context(context_val):
        if isinstance(context_val, list):
            return "\n".join([item.page_content if hasattr(item, "page_content") else str(item) for item in context_val])
        return str(context_val)
    
    df["ContextText"] = df["context"].apply(lambda x: _flatten_context(x))

    def _evaluate_row(row):
        relevance_score = relevance_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["ContextText"]
        )
        accuracy_score = accuracy_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["target_answer"]
        )
        return relevance_score, accuracy_score
    
    df[["Relevance", "Accuracy"]] = df.apply(lambda row: pd.Series(_evaluate_row(row)), axis=1)
    
    df["RelevanceScore"] = df["Relevance"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    df["AccuracyScore"] = df["Accuracy"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    
    # 平均スコアを計算
    avg_relevance = df["RelevanceScore"].mean()
    avg_accuracy = df["AccuracyScore"].mean()
    
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Accuracy: {avg_accuracy:.2f}")

    return df

In [15]:
results = evaluate_response(rag_eval_df)
results 

Average Relevance: 0.14
Average Accuracy: 0.29


,question,target_answer,response,context,ContextText,Relevance,Accuracy,RelevanceScore,AccuracyScore
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,I don't know.,[page_content='オープンイノベーション促進税制において、スタートアップ企業の株...,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,具体的な条件として、企業が主に「国内で」、「自ら」開発した知的財産権（特許権等、AI関連のプ...,[page_content='イノベーション拠点税制における所得控除について、企業が主に「国...,イノベーション拠点税制における所得控除について、企業が主に「国内で」、「自ら」開発した知財に...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,「カーブアウト加速等支援事業」の主な目的は、事業会社に蓄積されている技術を活用し、新たな会社...,[page_content='「カーブアウト加速等支援事業」' metadata={'pag...,「カーブアウト加速等支援事業」\nカーブアウト加速等支援事業,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",0,1
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,"I'm sorry, I don't know.",[],,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,"I'm sorry, but the specific information about ...",[page_content='- 2023年１月、産業革新投資機構（JIC）の子会社であるベ...,- 2023年１月、産業革新投資機構（JIC）の子会社であるベンチャー・グロース・インベスト...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
5,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間の違いに加えて、要件や...,スタートアップ支援資金は融資限度額が20億円で直接貸付、返済期間は20年以内、要件はJVCA...,"I'm sorry, but I don't have the specific infor...",[],,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
6,宇宙戦略基金の設立と関連する技術開発テーマの具体的な支援分野について説明し、各分野間でどのよ...,宇宙戦略基金は、民間企業や大学、スタートアップ、国立研究機関に対して10年間にわたる研究開発...,"I'm sorry, I don't have the specific informati...",[page_content='「宇宙技術戦略等を踏まえて、我が国として推進すべき技術開発テー...,「宇宙技術戦略等を踏まえて、我が国として推進すべき技術開発テーマ（最大10年）を設定。「輸送...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0


なんかいまいちだな。